# Transfer Learning for time series

https://www.youtube.com/watch?v=izcmIemT-rU&t=1s

In [ ]:
!pip install darts[all]

In [ ]:
from darts.dataprocessing.transformers import Scaler
from darts import TimeSeries
from darts.models import NaiveSeasonal, ARIMA, ExponentialSmoothing, NBEATSModel
from darts.metrics import smape, mase, mape, rmse

import tqdm.notebook as tq

import seaborn as sns

import os
import time
import random
import pandas as pd
import numpy as np

import torch
from torch import nn


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM,GRU,SimpleRNN, RNN, Input, Bidirectional
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from sklearn.model_selection import GroupKFold
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from datetime import timedelta

from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler

from typing import List, Tuple, Dict

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') 

import warnings
warnings.simplefilter(action='ignore', category= FutureWarning)

In [ ]:
class CFG:
    seed = 42    
    img_dim1 = 20
    img_dim2 = 10
    horizon = 12
    min_size = 48
    start_point = '2014-01-01'
    which_art = 'FOODS_3_819'
    cutoff_point = '2016-05-08'
    lookback = 120
    lookahead = 14
    nepochs = 10
    bsize = 64
    
# adjust the parameters for displayed figures    
plt.rcParams.update({'figure.figsize': (CFG.img_dim1,CFG.img_dim2)})   

In [ ]:
# function definitions
def plot_series(df, xname):
    xd = df.loc[df.airlineid == xname ]
    airname = xd['Description'].iat[0]
    xd[['data_dte', 'Total']].set_index('data_dte').plot(title = airname, xlabel = '')
    


<a id="section-groundwork"></a>
# Groundwork


Our reference series will be the data from the U.S. International Air Passenger and Freight Statistics Report on air traffic to and from the US: https://www.kaggle.com/datasets/parulpandey/us-international-air-traffic-data

In [ ]:
xdat = pd.read_csv('../input/us-international-air-traffic-data/International_Report_Passengers.csv',
                 usecols = ['data_dte', 'airlineid', 'Total']
                  )
xdat['data_dte'] = pd.to_datetime(xdat['data_dte'])
xdat.head(5)

In [ ]:
# we will aggregate to total passenger count per day
xmat = xdat.groupby(['data_dte', 'airlineid'])['Total'].sum().reset_index()
xmat['data_dte'] = pd.to_datetime(xmat['data_dte'])
xmat.sort_values(by = 'data_dte', inplace = True)


# map the identifiers to proper names
xdict = pd.read_csv('../input/airline-codes-mapping/L_AIRLINE_ID.csv')
xmat = pd.merge(left = xmat, right = xdict, left_on = 'airlineid', right_on = 'Code')
xmat.drop('Code', axis = 1, inplace = True)
xmat.head(5)

In [ ]:
# list of names 
names_list = xmat.airlineid.unique()

In [ ]:
plot_series(xmat, names_list[0])

In [ ]:
plot_series(xmat, names_list[1])

In [ ]:
plot_series(xmat, names_list[31])

We can clearly see that the series exhibit quite different dynamics (including number of observations). Let's prepare the our time series with a clean train / test split (for a refresher about *proper* way to do cross-validation for time series, please check out [Episode 10](https://www.kaggle.com/code/konradb/ts-10-validation-methods-for-time-series)). 


The approach we follow in this section is inspired by the AMLD workshop on transfer and meta learning https://appliedmldays.org/events/amld-epfl-2022/workshops/forecasting-meta-learning


In [ ]:

all_series = []
all_names = []

for (ii, name) in enumerate(names_list):
    xd = xmat.loc[xmat.airlineid == name]
    
    # for speed sake we will downsample to monthly freq
    x2 = xd[['data_dte', 'Total']].set_index('data_dte').resample('1M').sum().clip(lower = 1)
    xseries = TimeSeries.from_dataframe(x2, freq = 'M')
    
    if len(xseries) > CFG.min_size:
        
        all_series.append(xseries)
        all_names.append(name)

        

In [ ]:
# train / test split

air_train = [s[: - CFG.horizon] for s in all_series]
air_test = [s[ - CFG.horizon:] for s in all_series]

In [ ]:
# normalize the data 
print('scaling series...')
scaler_air = Scaler(scaler=MaxAbsScaler())
air_train_scaled: List[TimeSeries] = scaler_air.fit_transform(air_train)
air_test_scaled: List[TimeSeries] = scaler_air.transform(air_test)

In [ ]:
# helper functions from the official Darts tutorial

# evaluate a set of forecasts for multiple series (in darts format)    
def eval_forecasts(pred_series: List[TimeSeries], 
                   test_series: List[TimeSeries]) -> List[float]:
  
    print('computing sMAPEs...')
    smapes = smape(test_series, pred_series)
    mean, std = np.round(np.mean(smapes),4), np.round(np.std(smapes),4)
    print('Avg sMAPE: %.3f +- %.3f' % (mean, std))
    plt.figure(figsize = (CFG.img_dim1,CFG.img_dim2), dpi=144)
    plt.hist(smapes, bins=50)
    plt.ylabel('Count')
    plt.xlabel('sMAPE')
    plt.show()
    plt.close()
    return smapes


def eval_local_model(train_series: List[TimeSeries], 
                     test_series: List[TimeSeries], 
                     horizon,
                     model_cls, 
                     **kwargs) -> Tuple[List[float], float]:
    preds = []
    start_time = time.time()
    for series in tq.tqdm(train_series):
        model = model_cls(**kwargs)
        model.fit(series)
        pred = model.predict(n=horizon)
        preds.append(pred)
    elapsed_time = time.time() - start_time
    
    smapes = eval_forecasts(preds, test_series)
    return smapes, elapsed_time

Let's try a simple baseline: repeating the annual seasonal pattern

In [ ]:
ns_last_smapes, ns_last_elapsed_time  = eval_local_model(air_train_scaled, air_test_scaled, 
                                                                                 CFG.horizon, NaiveSeasonal, K= 12)

What about something a little more bespoke?

In [ ]:
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter('ignore', UserWarning)


arima_smapes, arima_elapsed_time = eval_local_model(air_train_scaled, air_test_scaled, CFG.horizon, 
                                                                                 ARIMA, p=12, d=1, q=1)

In [ ]:
es_smapes, es_elapsed_time = eval_local_model(air_train, air_test,  CFG.horizon, 
                                              ExponentialSmoothing)


Since the individual series tend to be rather short, we focus on vintage models here: the results above give us an indication what to expect; beating `NaiveSeasonal` is not trivial, apparently. 


Let's start exploiting the joint structure in the data, starting with fitting a single model for all series together - effectively first step towards transfer learning.

<a id="section-joint"></a>
# Joint model

For our joint model, we will use `NBEATS`, which you might remember from [Episode 9](https://www.kaggle.com/code/konradb/ts-9-hybrid-methods): it's a deep learning architecture, built around something of a hierarchical aggregation of MLP.

In [ ]:
from darts.utils.losses import SmapeLoss

# Slicing hyper-params:
IN_LEN = 24
OUT_LEN = 12

# Architecture hyper-params:
NUM_STACKS = 18
NUM_BLOCKS = 3
NUM_LAYERS = 3
LAYER_WIDTH = 180
COEFFS_DIM = 6
LOSS_FN = SmapeLoss()

# Training settings:
LR = 5e-4
BATCH_SIZE = 1024
NUM_EPOCHS = 5


In [ ]:
# reproducibility
np.random.seed(42)
torch.manual_seed(42)
start_time = time.time()
nbeats_model_air = NBEATSModel(input_chunk_length=IN_LEN,
                               output_chunk_length=OUT_LEN, 
                               batch_size=BATCH_SIZE,
                               nr_epochs_val_period=1,
                               num_stacks=NUM_STACKS,
                               num_blocks=NUM_BLOCKS,
                               num_layers=NUM_LAYERS,
                               layer_widths=LAYER_WIDTH,
                               expansion_coefficient_dim=COEFFS_DIM,
                               loss_fn=LOSS_FN,
                               optimizer_kwargs={'lr': LR},
                               pl_trainer_kwargs={"enable_progress_bar": True, 
                                                  "accelerator": "gpu",
                                                  "gpus": -1,
                                                  "auto_select_gpus": True},
                               )

nbeats_model_air.fit(air_train_scaled, num_loader_workers=1, epochs=NUM_EPOCHS)

nb_preds = nbeats_model_air.predict(series = air_train_scaled, n = CFG.horizon)
nbeats_elapsed_time = time.time() - start_time
nbeats_smapes = eval_forecasts(nb_preds, air_test_scaled)

In [ ]:
def smapes_boxplot(method_to_smapes: Dict[str, List[float]], title: str):
    method_names = []
    smapes = []
    for curr_method_name, curr_smapes in method_to_smapes.items():        
        method_names += [curr_method_name] * len(curr_smapes)
        smapes += curr_smapes
    smapes_df = pd.DataFrame({'Method': method_names, 'sMAPE': smapes})
    plt.figure(figsize=(CFG.img_dim1,CFG.img_dim2), dpi=144)
    ax = sns.boxplot(x="Method", y="sMAPE", data=smapes_df)
    ax.grid(False)
    # Display median score on each box
    medians = smapes_df.groupby(['Method'])['sMAPE'].median().round(decimals=2)
    vertical_offset = smapes_df['sMAPE'].median() * 0.1
    for xtick, name in enumerate(method_to_smapes.keys()):
        
        ax.text(xtick, medians[name] + vertical_offset, medians[name], 
                  horizontalalignment='center', size='x-small', color='w', weight='semibold')
        plt.xticks(rotation=90) 
        plt.title(title) 
        plt.show()
        plt.close()

def elapsed_time_barplot(method_to_elapsed_times: Dict[str, float], title: str):
    elapsed_times_df = pd.DataFrame({'Method': method_to_elapsed_times.keys(), 'Elapsed time [s]': method_to_elapsed_times.values()})
    ax = plt.figure(figsize=(CFG.img_dim1,CFG.img_dim2), dpi=144)
    sns.barplot(x="Method", y="Elapsed time [s]", data=elapsed_times_df)
    plt.xticks(rotation=90) 
    plt.title(title) 
    plt.show()
    plt.close()


In [ ]:
smapes  = pd.DataFrame.from_dict( {'exp smoothing': es_smapes, 'naive': ns_last_smapes,
                                       'ARIMA': arima_smapes,
                                   'NBEATS': nbeats_smapes})

smapes.boxplot()

In [ ]:
etimes = pd.DataFrame({'Model': ['exp smoothing', 'naive', 'ARIMA', 'NBEATS'],
                        'Time': [es_elapsed_time, ns_last_elapsed_time, arima_elapsed_time,
                                nbeats_elapsed_time]})
bargraph = etimes.plot(kind = 'bar',x = 'Model', y = 'Time', color = ['b', 'g', 'r', 'c'])



A joint model can offer performance on part with the local ones, if not the best of the bunch - although it less dispersed in its results. Can we improve the situation with more data? 


<a id="section-tl1"></a>
# TL: pretraining a time series model

CV models went wild with the ability to pretrain on large amounts of images (Imagenet, Instagram), similar pattern emerged with NLP;  a simple adaptation of this approach to time series involves using a large collection of time series to train our model and then using it for prediction $\rightarrow$ the usual point about downstream task / labeling is moot here.

In [ ]:
xdat_m4 = pd.read_csv('../input/m4-forecasting-competition-dataset/Monthly-train.csv')
xdat_m4.drop('V1', axis = 1, inplace = True)
xdat_m4.head(3)

We proceed in a similar fashion as with the "baseline" model above: 

In [ ]:
# create a list of series
m4_series = []

for ii in range(xdat_m4.shape[0]):
    
    # drop the missing values - this can be improved by only dropping contiguous blocks
    xd = xdat_m4.iloc[ii].dropna()

    xseries = TimeSeries.from_values(xd.values)
    
    if len(xseries) > 172:        
        m4_series.append(xseries)

# train / test split

m4_train = [s[: - 72] for s in m4_series]
m4_test = [s[ - 72:] for s in m4_series]



In [ ]:
# normalize the data 
scaler_m4 = Scaler(scaler=MaxAbsScaler())
m4_train_scaled: List[TimeSeries] = scaler_m4.fit_transform(m4_train)
m4_test_scaled: List[TimeSeries] = scaler_m4.transform(m4_test)

In [ ]:
from darts.utils.losses import SmapeLoss

# Slicing hyper-params:
IN_LEN = 24
OUT_LEN = 12
# Architecture hyper-params:
NUM_STACKS = 18
NUM_BLOCKS = 3
NUM_LAYERS = 3
LAYER_WIDTH = 180
COEFFS_DIM = 6
LOSS_FN = SmapeLoss()
# Training settings:
LR = 5e-4
BATCH_SIZE = 1024
MAX_SAMPLES_PER_TS = 64
NUM_EPOCHS = 5


In [ ]:
# reproducibility
np.random.seed(42)
torch.manual_seed(42)

nbeats_model_m4 = NBEATSModel(input_chunk_length=IN_LEN,
                               output_chunk_length=OUT_LEN, 
                               batch_size=BATCH_SIZE,
                               nr_epochs_val_period=1,
                               num_stacks=NUM_STACKS,
                               num_blocks=NUM_BLOCKS,
                               num_layers=NUM_LAYERS,
                               layer_widths=LAYER_WIDTH,
                               expansion_coefficient_dim=COEFFS_DIM,
                               loss_fn=LOSS_FN,
                               optimizer_kwargs={'lr': LR},
                               pl_trainer_kwargs={"enable_progress_bar": True, 
                                                  "accelerator": "gpu",
                                                  "gpus": -1,
                                                  "auto_select_gpus": True},
                               )



In [ ]:
nbeats_model_m4.fit(m4_train_scaled,
                    num_loader_workers=1,
                    max_samples_per_ts=MAX_SAMPLES_PER_TS,
                    epochs=NUM_EPOCHS, val_series = m4_test_scaled,
                    verbose = True
                   )


In [ ]:
start_time = time.time()
nbm4_preds = nbeats_model_m4.predict(series = air_train_scaled, n = CFG.horizon)
nbm4_elapsed_time = time.time() - start_time
nbm4_smapes = eval_forecasts(nbm4_preds, air_test_scaled)

In [ ]:
smapes  = pd.DataFrame.from_dict( {'exp smoothing': es_smapes, 'naive': ns_last_smapes,
                                       'ARIMA': arima_smapes,   'NBEATS': nbeats_smapes,
                                       'NBEATS-M4': nbm4_smapes
                                    })

smapes.boxplot()

As you can see from the graph above, NBEATS pretrained on M4 is comparable in performance with the individual models per series - **despite the fact it was not trained on any of the airline data**. What about execution time?

In [ ]:
etimes = pd.DataFrame({'Model': ['exp smoothing', 'naive', 'ARIMA', 'NBEATS', 'NBEATS-M4'],
                        'Time': [es_elapsed_time, ns_last_elapsed_time, arima_elapsed_time,
                                nbeats_elapsed_time, nbm4_elapsed_time]})
bargraph = etimes.plot(kind = 'bar',x = 'Model', y = 'Time', color = ['b', 'g', 'r', 'c', 'y'])

Not only does it achieve performance (which can undoubtedly be improved with longer training / hyperparameters tuning), but it's also blazing fast - we don't need to train anything specifically for this application. 


<a id="section-tl2"></a>
# TL: the tinkering 

In the previous section we re-used the same architecture fitted on a dataset with the same frequency as the target one. While those contraints are not very harsh, let us see how we can become more flexible in our transfer learning adventure. We will use the data from the [M5 Competition](https://www.kaggle.com/c/m5-forecasting-accuracy) competition  and recycle code bits from [Episode 6](https://www.kaggle.com/code/konradb/ts-6-deep-learning-for-ts-rnn-and-friends/):

In [ ]:
## prepare the data
calendar_df = pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv', parse_dates=['date'])
calendar_df = calendar_df.loc[:, ['date', 'wm_yr_wk', 'd']]
df = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_evaluation.csv')
df = df.loc[df.item_id== CFG.which_art]
df_T = df.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])
df_T.drop(columns=['id'], inplace=True)

sales_df = df_T.merge(calendar_df, left_on='variable', right_on='d', how='left')
sales_df.rename(columns={'value': 'sales_qty'}, inplace=True)
df = sales_df.loc[sales_df.date >= CFG.start_point, ['date', 'store_id', 'sales_qty']]
df['state_id'] = df['store_id'].str[:2]

# create the long format matrix: individual stores
df_ind = df.groupby(['date', 'store_id'])[['sales_qty']].sum()
df_ind.reset_index(inplace=True)
df_ind = df_ind.T.reset_index(drop=True).T
df_ind.columns = ['ds', 'unique_id', 'sales']

df_ind.sort_values(by = ['unique_id','ds'], inplace = True)



In [ ]:
## split into training and holdout
xdat = df_ind.pivot(index='ds', columns='unique_id', values='sales')

xtrain, xtest = xdat.iloc[:-(2 * CFG.lookahead + CFG.lookback)], xdat.iloc[-(2 * CFG.lookahead + CFG.lookback):]
x0, x1 = xtrain.iloc[:-(2 * CFG.lookahead + CFG.lookback)], xtrain.iloc[- (2 * CFG.lookahead + CFG.lookback):]

# scale / normalize the columns
xcols = xtrain.columns

RS = RobustScaler()
RS.fit(xtrain)

xtrain = RS.fit_transform(xtrain.to_numpy())
xtrain = pd.DataFrame(xtrain, columns= xcols)

xtest = RS.transform(xtest.to_numpy())
xtest = pd.DataFrame(xtest, columns= xcols)

In [ ]:
## format into LSTM-friendly (so that it also works with the M4 data format)
def create_dataset(dataset, look_back, look_ahead):
    xdat, ydat = [], []
    for f in dataset.columns:
        for i in range(len(dataset) - look_back -look_ahead):
            xdat.append(dataset.iloc[i:i+ look_back][f])
            ydat.append(dataset.iloc[i+ look_back : i + look_back + look_ahead][f])
    xdat, ydat = np.array(xdat), np.array(ydat).reshape(-1,look_ahead)
    xdat = xdat.reshape(-1, look_back, 1)
    return xdat.astype(np.float32), ydat.astype(np.float32)

In [ ]:
x0, y0 = create_dataset(dataset = x0, look_back = CFG.lookback, look_ahead = CFG.lookahead)
x1, y1 = create_dataset(dataset = x1, look_back = CFG.lookback, look_ahead = CFG.lookahead)

xtrain, ytrain = create_dataset(dataset = xtrain, look_back = CFG.lookback, look_ahead = CFG.lookahead)
xtest, ytest = create_dataset(dataset = xtest, look_back = CFG.lookback, look_ahead = CFG.lookahead)

Try a simple architecture for the model we will crash test across datasets:

In [ ]:
def create_model(out_dim):    
    
    model=Sequential()
    model.add(SimpleRNN(10,input_shape= [None,1], return_sequences = True))
    model.add(SimpleRNN(10,input_shape= [None,1]))
    model.add(Dense(out_dim))
    
    model.compile(loss='mean_squared_error',optimizer='adam')
    return model
    
base_model = create_model(out_dim = CFG.lookahead)

In [ ]:
base_model.summary()

In [ ]:
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)

base_model.fit(x0, y0, validation_data=(x1, y1), 
                  epochs = CFG.nepochs, batch_size = CFG.bsize, callbacks=[ early_stop])

The main reason for this part is to get a feel how many epochs are necessary - seems like 10 are fine. Let's fit the model on the complete dataset now: 

In [ ]:
# fit on complete training data
base_model = create_model(out_dim = CFG.lookahead)
base_model.fit(xtrain, ytrain,  epochs = CFG.nepochs, batch_size = CFG.bsize)

Predict on the holdout set, with necessary reshaping (to ensure inverse normalization works):

In [ ]:
# predict on holdout
ypred_base = base_model.predict(xtest)

ypred_base = np.reshape(ypred_base.T, (-1,10), order = 'F')
ypred_base = RS.inverse_transform(ypred_base)

ytest = np.reshape(ytest.T, (-1,10), order = 'F')
ytest = RS.inverse_transform(ytest)

In [ ]:
from sklearn.metrics import mean_squared_error as mse


def my_rmse(x,y):
    return(np.round( np.sqrt(mse(x,y)) ,4))

base_error = my_rmse(ytest, ypred_base)
print(base_error)

This gives us the base error rate. What if we train the model on the M4 dataset first (using daily series so we capture similar types of patterns)?

In [ ]:
m4dat = pd.read_csv('../input/m4-forecasting-competition-dataset/Daily-train.csv', nrows= 500)
m4dat.drop('V1', axis = 1, inplace = True)
# there are more elegant ways, I know...
m4dat.dropna(axis = 1, how = 'any', inplace = True)
print(m4dat.shape)

xcols2 = m4dat.columns

RS2 = RobustScaler()
RS2.fit(m4dat)

m4dat = RS2.fit_transform(m4dat.to_numpy())
m4dat = pd.DataFrame(m4dat, columns= xcols2)

m4_model = create_model(out_dim = CFG.lookahead)

m4_x, m4_y = create_dataset(dataset = m4dat, look_back = CFG.lookback, look_ahead = CFG.lookahead)

In [ ]:
m4_model.fit(m4_x, m4_y,  epochs = 5, batch_size = CFG.bsize)

In [ ]:
# predict on holdout
ypred_m4 = m4_model.predict(xtest)

ypred_m4 = np.reshape(ypred_m4.T, (-1,10), order = 'F')
ypred_m4 = RS.inverse_transform(ypred_m4)

In [ ]:
pretrain_error = my_rmse(ytest, ypred_m4)
print(pretrain_error)

Unlike the airline dataset case in the previous section, this is solidly worse than the "baseline" model - reasons can include architecture simplicity (we just don't learn enough with two small layers) or the fact that difference in domains does actually matter in this instance. Proper investigation is outside the scope of this notebook, so let's instead do something we have not tried and finetune the pretrained model (~ moving from zero- to few-shot learning).

In [ ]:
m4_model.fit(xtrain, ytrain,
                  epochs = 5, batch_size = CFG.bsize)

In [ ]:
# predict on holdout
ypred_m4f = m4_model.predict(xtest)

ypred_m4f = np.reshape(ypred_m4f.T, (-1,10), order = 'F')
ypred_m4f = RS.inverse_transform(ypred_m4f)

In [ ]:
finetune_error = my_rmse(ytest, ypred_m4f)
print(my_rmse(ytest, ypred_m4f))

In [ ]:
print('baseline: ' + str(base_error))
print('pretrain: ' + str(pretrain_error))
print('finetune: ' + str(finetune_error))


Long story short, pretraining on a large dataset helps for time series :-) 

<a id="section-tl3"></a>
# TL: more tinkering 

After the encouraging results of the preceding section I started wondering: was it just a fluke? The simplest way to address such question is to try an break something - in this instance, I decided to use M4 data but from a different frequency. My intuition was that the model would try to capture different patterns. 

In [ ]:
m4dat_m = pd.read_csv('../input/m4-forecasting-competition-dataset/Monthly-train.csv', nrows= 500)
m4dat_m.drop('V1', axis = 1, inplace = True)
m4dat_m.dropna(axis = 1, how = 'any', inplace = True)
print(m4dat_m.shape)

In [ ]:
xcols_m = m4dat_m.columns

RS_m = RobustScaler()
RS_m.fit(m4dat_m)

m4dat_m = RS_m.fit_transform(m4dat_m.to_numpy())
m4dat_m = pd.DataFrame(m4dat_m, columns= xcols_m)

m4_model_m = create_model(out_dim = CFG.lookahead)

m4_x_m, m4_y_m = create_dataset(dataset = m4dat_m, look_back = CFG.lookback, look_ahead = CFG.lookahead)

m4_model_m.fit(m4_x_m, m4_y_m,  epochs = 5, batch_size = CFG.bsize)

In [ ]:
# predict on holdout
ypred_m4_m = m4_model_m.predict(xtest)

ypred_m4_m = np.reshape(ypred_m4_m.T, (-1,10), order = 'F')
ypred_m4_m = RS.inverse_transform(ypred_m4_m)

pretrain_error_m = my_rmse(ytest, ypred_m4_m)
print(pretrain_error_m)

In [ ]:
m4_model_m.fit(xtrain, ytrain,
                  epochs = 5, batch_size = CFG.bsize)

# predict on holdout
ypred_m4_mf = m4_model_m.predict(xtest)

ypred_m4_mf = np.reshape(ypred_m4_mf.T, (-1,10), order = 'F')
ypred_m4_mf = RS.inverse_transform(ypred_m4_mf)

finetune_error_m = my_rmse(ytest, ypred_m4_mf)
print(finetune_error_m)


In [ ]:
print('baseline: ' + str(base_error))
print('pretrain: ' + str(pretrain_error))
print('pretrain_m: ' + str(pretrain_error_m))
print('finetune: ' + str(finetune_error))
print('finetune_m: ' + str(finetune_error_m))